<a href="https://colab.research.google.com/github/m-manuelmussa/Molecular-Docking/blob/main/Preparo_de_Estruturas_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparo de estruturas 3D**

*@Micliete_Mussa*

# **PIPELINE**
1. Instalação de dependências chave;
2. Importação de frameworks necessários;
3. Conversão de formato CIF para PDB;
4. Criação de diretórios;
3. Remoção de Heteroatomos (excepto água);
4. Protonação (pH=7.4);
5. Verificação do resultado
8. Compactação dos ficheiros PQR
9. Download do Zip

[Documentação PDB2PQR ](https://pdb2pqr.readthedocs.io/en/latest/)

## **1. Instalação de dependências chave**

In [ ]:
# Instalação dependências
!apt-get install -y openbabel #Para conversão de formato das estruturas
!pip install pdb2pqr #Para o preparo propriamente dito

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openbabel is already the newest version (3.1.1+dfsg-6ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


## **2. Importação de frameworks necessários**

In [ ]:
import os #Para manipulação arquivos, diretórios e caminhos.
import subprocess #Para execução comandos do terminal
import shutil #Para Compactação de pastas
from google.colab import files #Para controle de pastas no ambiente Colab

## **3. Conversão de formato CIF para PDB**

In [ ]:
folder_path = "/content"
pdb_folder = os.path.join(folder_path, "pdb_files")
os.makedirs(pdb_folder, exist_ok=True)

# Lista que vai armazenar os caminhos dos PDBs convertidos
pdb_files = []

# Converter todos os CIF para PDB
for cif_file in os.listdir(folder_path):
    if cif_file.endswith(".cif"):
        cif_path = os.path.join(folder_path, cif_file)
        pdb_name = os.path.splitext(cif_file)[0] + ".pdb"
        pdb_path = os.path.join(pdb_folder, pdb_name)
        # Converter via OpenBabel
        os.system(f"obabel {cif_path} -O {pdb_path}")
        # Adicionar caminho do PDB gerado na lista
        pdb_files.append(pdb_path)

## **4. Criação de diretórios**

In [ ]:
# === Diretórios ===
base_folder = os.path.dirname(pdb_files[0])
output_clean_folder = os.path.join(base_folder, "cleaned_pdbs")
output_pqr_folder = os.path.join(base_folder, "pqr_pdb2pqr")
os.makedirs(output_clean_folder, exist_ok=True)
os.makedirs(output_pqr_folder, exist_ok=True)

## **5. Remoção de Heteroatomos**

In [ ]:
cleaned_pdb_files = []
generated_pqr_files = []

for pdb_path in pdb_files:
    pdb_name = os.path.basename(pdb_path)
    name = os.path.splitext(pdb_name)[0]

    # --- Limpeza: mantém proteína e água, remove outros HETATM ---
    clean_pdb = os.path.join(output_clean_folder, f"{name}_clean.pdb")
    with open(pdb_path, "r") as fin, open(clean_pdb, "w") as fout:
        for line in fin:
            if line.startswith("ATOM"):
                fout.write(line)
            elif line.startswith("HETATM") and "HOH" in line:
                fout.write(line)  # mantém apenas água
    cleaned_pdb_files.append(clean_pdb)

## **6. Protonação (pH=7.4)**

In [ ]:
generated_pqr_files = []

for clean_pdb in cleaned_pdb_files:
    name = os.path.splitext(os.path.basename(clean_pdb))[0].replace("_clean", "")
    pqr_out = os.path.join(output_pqr_folder, f"{name}.pqr")

    cmd = [
        "pdb2pqr",
        "--ff", "AMBER",
        "--with-ph", "7.4",
        clean_pdb,
        pqr_out
    ]

    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if result.returncode == 0 and os.path.exists(pqr_out):
        generated_pqr_files.append(pqr_out)

## **7. Verificação do resultado**

In [ ]:
any(" H" in line[12:16] for line in open(pqr_out))

True

## **8. Compactação dos ficheiros PQR**

In [ ]:
# Caminho da pasta com os PQRs
pqr_folder = output_pqr_folder
zip_path = "/content/pqr_files.zip"  # arquivo zip que será criado

# Compacta todos os arquivos da pasta
shutil.make_archive(base_name="/content/pqr_files", format='zip', root_dir=pqr_folder)
print("Pasta compactada em:", zip_path)

Pasta compactada em: /content/pqr_files.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **9. Download do Zip**

In [ ]:
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>